In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Load the dataset
data = pd.read_csv("../data/model_data.csv")
data.head()

,age_bracket,bmi_bracket,sex,children,smoker,region,charges
0,1,4,0,0,1,3,16884.92400
1,0,1,1,1,0,2,1725.55230
2,2,1,1,3,0,2,4449.46200
3,3,0,1,0,0,1,21984.47061
4,3,4,1,0,0,1,3866.85520


In [3]:
#Train-Test Split
from sklearn.model_selection import train_test_split

# Baseline model features
X_base = data[['smoker', 'age_bracket']]
y = data['charges']

# Split into train/test
X_train_base, X_test_base, y_train, y_test = train_test_split(X_base, y, test_size=0.2, random_state=42)

In [4]:
#Baseline Linear Regression Model (Smoker + Age Bracket)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Train baseline model
base_model = LinearRegression()
base_model.fit(X_train_base, y_train)

# Predict
y_train_pred_base = base_model.predict(X_train_base)
y_test_pred_base = base_model.predict(X_test_base)

# Evaluate performance
r2_train_base = r2_score(y_train, y_train_pred_base)
r2_test_base = r2_score(y_test, y_test_pred_base)
mae_base = mean_absolute_error(y_test, y_test_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_test, y_test_pred_base))

print("📊 Baseline Model (Smoker + Age Bracket):")
print(f"R² Train: {r2_train_base:.3f}")
print(f"R² Test:  {r2_test_base:.3f}")
print(f"MAE: {mae_base:.2f}")
print(f"RMSE: {rmse_base:.2f}")

📊 Baseline Model (Smoker + Age Bracket):
R² Train: 0.710
R² Test:  0.752
MAE: 4009.12
RMSE: 6203.33


In [ ]:
#Full Linear Regression Model (All Features)
# Full model features (everything except target)
X_full = data.drop('charges', axis=1)

# Split data
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

# Train full model
full_model = LinearRegression()
full_model.fit(X_train_full, y_train)

# Predict
y_train_pred_full = full_model.predict(X_train_full)
y_test_pred_full = full_model.predict(X_test_full)

# Evaluate
r2_train_full = r2_score(y_train, y_train_pred_full)
r2_test_full = r2_score(y_test, y_test_pred_full)
mae_full = mean_absolute_error(y_test, y_test_pred_full)
rmse_full = np.sqrt(mean_squared_error(y_test, y_test_pred_full))

print("\n📊 Full Model (All Features):")
print(f"R² Train: {r2_train_full:.3f}")
print(f"R² Test:  {r2_test_full:.3f}")
print(f"MAE: {mae_full:.2f}")
print(f"RMSE: {rmse_full:.2f}")


📊 Full Model (All Features):
R² Train: 0.713
R² Test:  0.757
MAE: 3919.83
RMSE: 6145.79


In [13]:
#Compare Model Performance
comparison = pd.DataFrame({
    'Model': ['Baseline (Smoker + Age Bracket)', 'Full Model (All Features)'],
    'R² Train': [r2_train_base, r2_train_full],
    'R² Test': [r2_test_base, r2_test_full],
    'MAE': [mae_base, mae_full],
    'RMSE': [rmse_base, rmse_full]
})

display(comparison)

,Model,R² Train,R² Test,MAE,RMSE
0,Baseline (Smoker + Age Bracket),0.709666,0.752131,4009.118418,6203.333931
1,Full Model (All Features),0.712622,0.756709,3919.829703,6145.786727


In [7]:
#Check Multicollinearity (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select only the independent variables (exclude the target)
X = data.drop('charges', axis=1)

# Compute VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display VIF
print(vif_data)

       Feature       VIF
0  age_bracket  2.558196
1  bmi_bracket  2.273807
2          sex  1.817245
3     children  1.684954
4       smoker  1.234649
5       region  2.322247


The VIF values are within 1-5 indicating low multicollinearity

In [ ]:
#Show Feature importance
X = data.drop('charges', axis=1)
y = data['charges']

# Fit model
lr = LinearRegression()
lr.fit(X, y)

# Get coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr.coef_
}).sort_values(by='Coefficient', ascending=False)

coefficients

,Feature,Coefficient
4,smoker,23804.807458
0,age_bracket,1385.615511
3,children,524.795853
2,sex,62.031963
5,region,-80.787556
1,bmi_bracket,-199.182148


In [ ]:
#RandomForest Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Separate features and target
X = data.drop('charges', axis=1)
y = data['charges']

# Train-test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

# Evaluate
r2_train_rf = r2_score(y_train, y_train_pred_rf)
r2_test_rf = r2_score(y_test, y_test_pred_rf)
mae_rf = mean_absolute_error(y_test, y_test_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))

# Results summary
rf_results = pd.DataFrame({
    'Model': ['Random Forest'],
    'R² Train': [r2_train_rf],
    'R² Test': [r2_test_rf],
    'MAE': [mae_rf],
    'RMSE': [rmse_rf]
})

print(rf_results)

           Model  R² Train   R² Test          MAE         RMSE
0  Random Forest  0.951328  0.826978  2782.141656  5182.805935


In [12]:
#Compare All Models
comparison = pd.DataFrame({
    'Model': [
        'Baseline (Smoker + Age Bracket)',
        'Full Linear Regression',
        'Random Forest Regressor'
    ],
    'R² Train': [r2_train_base, r2_train_full, r2_train_rf],
    'R² Test': [r2_test_base, r2_test_full, r2_test_rf],
    'MAE': [mae_base, mae_full, mae_rf],
    'RMSE': [rmse_base, rmse_full, rmse_rf]
})

comparison

,Model,R² Train,R² Test,MAE,RMSE
0,Baseline (Smoker + Age Bracket),0.709666,0.752131,4009.118418,6203.333931
1,Full Linear Regression,0.712622,0.756709,3919.829703,6145.786727
2,Random Forest Regressor,0.951328,0.826978,2782.141656,5182.805935


In [11]:
#Modeling using XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Split data 
X = data.drop(columns='charges')
y = data['charges']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Train XGBoost Model
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)

# Predictions
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Evaluation metrics
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"R² Train: {r2_train:.6f}")
print(f"R² Test: {r2_test:.6f}")
print(f"MAE: {mae:.6f}")
print(f"RMSE: {rmse:.6f}")

R² Train: 0.918492
R² Test: 0.850865
MAE: 2839.535551
RMSE: 4811.765459


In [14]:
#Add to Comparison Table
model_comparison = pd.DataFrame({
    'Model': [
        'Baseline (Smoker + Age Bracket)',
        'Full Linear Regression',
        'Random Forest Regressor',
        'XGBoost Regressor'
    ],
    'R² Train': [r2_train_base, r2_train_full, r2_train_rf, r2_train],
    'R² Test': [r2_test_base, r2_test_full, r2_test_rf, r2_test],
    'MAE': [mae_base, mae_full, mae_rf, mae],
    'RMSE': [rmse_base, rmse_full, rmse_rf, rmse]
})

model_comparison

,Model,R² Train,R² Test,MAE,RMSE
0,Baseline (Smoker + Age Bracket),0.709666,0.752131,4009.118418,6203.333931
1,Full Linear Regression,0.712622,0.756709,3919.829703,6145.786727
2,Random Forest Regressor,0.951328,0.826978,2782.141656,5182.805935
3,XGBoost Regressor,0.918492,0.850865,2839.535551,4811.765459


The XGBoost Regressor emerged as the best-performing model, achieving a higher R² score on the test set and the lowest RMSE compared to the Random Forest model. Although the MAE difference between the two models is minimal, XGBoost demonstrates superior overall performance. Therefore, we will proceed with saving the XGBoost Regressor and use it to power our Streamlit app for making predictions.

In [16]:
#Save your model
import joblib

joblib.dump(xgb_model, "xgb_insurance_model.pkl")

['xgb_insurance_model.pkl']

In [18]:
#Load and test saved model
model = joblib.load("xgb_insurance_model.pkl")
print(model.get_booster().feature_names)

['age_bracket', 'bmi_bracket', 'sex', 'children', 'smoker', 'region']
